In [33]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [14]:
user_movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

In [15]:
user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
user_similarity = cosine_similarity(user_movie_matrix)

In [31]:
def recomendar_pelicula(user_id, user_movie_matrix, user_similarity, movies, top_n=5):
  # Obtener la similitud del usuario con todos los demás usuarios
  user_similarities = user_similarity[user_id-1]

  # Obtener los índices de los usuarios más similares (excluyendo al propio usuario)
  similar_users_indices = np.argsort(user_similarities)[::-1][1:]

  # Obtener las calificaciones de las películas de los usuarios más similares
  similar_users_ratings = user_movie_matrix.iloc[similar_users_indices]

  # Calcular la calificación predicha para el usuario objetivo para cada película
  predicted_ratings = similar_users_ratings.mean(axis=0)

  # Obtener las películas que el usuario objetivo aún no ha calificado
  unrated_movies = user_movie_matrix.iloc[user_id-1] == 0

  # Filtrar las calificaciones predichas para las películas no calificadas
  predicted_ratings = predicted_ratings[unrated_movies]

  # Ordenar las películas por calificación predicha en orden descendente
  recommended_movie_ids = predicted_ratings.sort_values(ascending=False).index.tolist()

  # Obtener los títulos de las películas recomendadas
  recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)].iloc[:top_n]['title'].tolist()

  return recommended_movies

In [32]:
# Ejemplo de uso:
user_id = 8
recommended_movies = recomendar_pelicula(user_id, user_movie_matrix, user_similarity, movies)

print("Películas recomendadas para el usuario", user_id, ":")
for movie in recommended_movies:
    print(movie)

Películas recomendadas para el usuario 8 :
Toy Story (1995)
Grumpier Old Men (1995)
Waiting to Exhale (1995)
Father of the Bride Part II (1995)
Heat (1995)
